In [ ]:
import warnings
warnings.filterwarnings(action='once')
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from exp.misc import *
from exp.ProcessData import *
from exp.PytorchModels import *
from exp.LearnerClass import *
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
import copy
from torchvision import transforms
import PIL.Image
from sklearn.metrics import roc_auc_score
import torchvision.transforms.functional as TF
from types import MethodType


In [ ]:
params=json_to_parameters('config.json')
num_folds=3
SEEDS=[220,432,8153]
add_seed=2
model_type='tf_efficientnet_b5_ns'
name_tamplate='image_mlps_cut_128_seed_fullv{}'

device = device_by_name('Tesla')

torch.cuda.set_device(device)

In [ ]:
test_df=pd.read_csv(params.path.data+'test.csv')
prepare_df(test_df)

In [ ]:
def my_metric(y_pred,y_true):
    preds=torch.sigmoid(torch.tensor(y_pred[:,1],dtype=torch.float32)).numpy()
    preds1=F.softmax(torch.tensor(y_pred,dtype=torch.float32),-1)[:,1].numpy()
    preds2=0.5*preds1/preds1.std()+0.5*preds/preds.std()
    return {'metric':-roc_auc_score(y_true[:,-1], preds1),'metric1':-roc_auc_score(y_true[:,-1], preds),
                                                                'metric2':-roc_auc_score(y_true[:,-1], preds2)}

In [ ]:
class MyLoss():
    def __init__(self,weight):
        self.weight=weight
    def __call__(self,y_pred,y_true):
        return self.weight*F.binary_cross_entropy_with_logits(y_pred[:,1],y_true[:,1].to(torch.float32),weight=torch.tensor(1.,device=device))+\
               (1-self.weight)*F.cross_entropy(y_pred,y_true[:,0])


In [ ]:
transform=transforms.Compose([HairTransform(1.0),transforms.RandomRotation(45),
                              transforms.RandomHorizontalFlip(p=0.5),transforms.RandomVerticalFlip(p=0.5),
                              transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.3, hue=0.1),
                              transforms.RandomResizedCrop((600,900), scale=(0.7, 1.1)),
                              transforms.ToTensor(),CutoutTransform(0.5,24,fill_const=False),
                              CutoutTransform(0.5,16,fill_const=False)])

transform_val=transforms.Compose([transforms.ToTensor()])


In [ ]:

ds=ImageDataset(params.path.test_jpg_small,test_df,transform=transform,meta_aug=0.1)
for SEED in SEEDS:
    for fold in range(num_folds): 
        torch.manual_seed(SEED+fold+add_seed)
        np.random.seed(SEED+fold+add_seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        model = get_model(model_type,8,extra=[8,8,[7,32]],mid_extra=128,mlps=[1024,256],dropout=0.4,bn=True).to(device)
        name=params.model_format.format(model_type,f'image_mlps_cut_128_seed_fullv_large{SEED}',1,fold)
        print (name)
        my_loss=MyLoss(0.9)
        #my_loss=my_one_loss
        learner = Learner(model,None,loss_func=my_loss,name=name,scheduler=None,device=device)
        learner.load_model(params.path.models)
        learner.init_amp()
        learner.model.last_linear=Noop()
        def new_get_x(self,batch):
            return batch
        learner.get_x=MethodType(new_get_x, learner)
        featuress=[]
        for i in notebook.tqdm(range(16)):
            features=learner.predict(ds,batch_size=32,num_workers=8)
            featuress.append(features)
        features=np.stack(featuress,0)
        with open(params.path.features+(name.split('.')[0]+'_test.pkl'),'wb') as f:
            pickle.dump(features,f,protocol=4)
            